# **DCGAN implementation of MNIST dataset**

Software Requirements:
python 3

library requirements 
1.   Torch
2.   Torchvision
3.   Matplotlib
4.   tensorboardX
5.    numpy
6.   Pillow


Running Instructions:
1. Install all libraries
2. Run the Code cell below 

DCGAN architecture Details

![DCGAN architecture details](https://raw.githubusercontent.com/znxlwm/pytorch-MNIST-CelebA-GAN-DCGAN/master/pytorch_DCGAN.png)


Generated images after a few epochs

![alt text](https://i.imgur.com/Vp1w3KS.png)


**References**:

1. [DCGAN Paper](https://arxiv.org/pdf/1511.06434.)
2. [MNIST- celebA implementation](https://github.com/znxlwm/pytorch-MNIST-CelebA-GAN-DCGAN)
3. [utility File, GAN tutorial](https://medium.com/ai-society/gans-from-scratch-1-a-deep-introduction-with-code-in-pytorch-and-tensorflow-cb03cdcdba0f)







logger utility class to save images , models and log tensorboard data


In [0]:
import os
import numpy as np
import errno
import torchvision.utils as vutils
from tensorboardX import SummaryWriter
from matplotlib import pyplot as plt
from IPython import display


# utility class to show image plots , save model, and log tensorboard Data
class Logger:
    def __init__(self, model_name, data_name):
        self.model_name = model_name
        self.data_name = data_name

        self.comment = '{}_{}'.format(model_name, data_name)
        self.data_subdir = '{}/{}'.format(model_name, data_name)

        # TensorBoard
        self.writer = SummaryWriter(comment=self.comment)
    
    def log(self, d_error, g_error, epoch, n_batch, num_batches):

        var_class = torch.autograd.Variable
        if type(d_error) == var_class:
            d_error = d_error.data.cpu().numpy()
        if type(g_error) == var_class:
            g_error = g_error.data.cpu().numpy()

        step = Logger._step(epoch, n_batch, num_batches)
        self.writer.add_scalar(
            '{}/D_error'.format(self.comment), d_error, step)
        self.writer.add_scalar(
            '{}/G_error'.format(self.comment), g_error, step)

    def log_images(self, images, num_images, epoch, n_batch, num_batches, format='NCHW', normalize=True):
        '''
        input images are expected in format (NCHW)
        '''
        if type(images) == np.ndarray:
            images = torch.from_numpy(images)

        if format == 'NHWC':
            images = images.transpose(1, 3)

        step = Logger._step(epoch, n_batch, num_batches)
        img_name = '{}/images{}'.format(self.comment, '')

        # Make horizontal grid from image tensor
        horizontal_grid = vutils.make_grid(
            images, normalize=normalize, scale_each=True)
        # Make vertical grid from image tensor
        nrows = int(np.sqrt(num_images))
        grid = vutils.make_grid(
            images, nrow=nrows, normalize=True, scale_each=True)

        # Add horizontal images to tensorboard
        self.writer.add_image(img_name, horizontal_grid, step)

        # Save plots
        self.save_torch_images(horizontal_grid, grid, epoch, n_batch)

    def save_torch_images(self, horizontal_grid, grid, epoch, n_batch, plot_horizontal=True):
        out_dir = './data/images/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)

        # Plot and save horizontal
        fig = plt.figure(figsize=(16, 16))
        plt.imshow(np.moveaxis(horizontal_grid.numpy(), 0, -1))
        plt.axis('off')
        if plot_horizontal:
            display.display(plt.gcf())
        self._save_images(fig, epoch, n_batch, 'hori')
        plt.close()

        # Save squared
        fig = plt.figure()
        plt.imshow(np.moveaxis(grid.numpy(), 0, -1))
        plt.axis('off')
        self._save_images(fig, epoch, n_batch)
        plt.close()

    def _save_images(self, fig, epoch, n_batch, comment=''):
        out_dir = './data/images/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)
        fig.savefig('{}/{}_epoch_{}_batch_{}.png'.format(out_dir,
                                                         comment, epoch, n_batch))

    def display_status(self, epoch, num_epochs, n_batch, num_batches, d_error, g_error, d_pred_real, d_pred_fake):

        var_class = torch.autograd.Variable
        if type(d_error) == var_class:
            d_error = d_error.data.cpu().numpy()[0]
        if type(g_error) == var_class:
            g_error = g_error.data.cpu().numpy()[0]
        if type(d_pred_real) == var_class:
            d_pred_real = d_pred_real.data
        if type(d_pred_fake) == var_class:
            d_pred_fake = d_pred_fake.data

        print('Epoch: [{}/{}], Batch Num: [{}/{}]'.format(
            epoch, num_epochs, n_batch, num_batches)
        )
        print('Discriminator Loss: {:.4f}, Generator Loss: {:.4f}'.format(d_error, g_error))
        print('D(x): {:.4f}, D(G(z)): {:.4f}'.format(d_pred_real.mean(), d_pred_fake.mean()))

    def save_models(self, generator, discriminator, epoch):
        out_dir = './data/models/{}'.format(self.data_subdir)
        Logger._make_dir(out_dir)
        torch.save(generator.state_dict(),
                   '{}/G_epoch_{}'.format(out_dir, epoch))
        torch.save(discriminator.state_dict(),
                   '{}/D_epoch_{}'.format(out_dir, epoch))

    def close(self):
        self.writer.close()

    # Private Functionality

    @staticmethod
    def _step(epoch, n_batch, num_batches):
        return epoch * num_batches + n_batch

    @staticmethod
    def _make_dir(directory):
        try:
            os.makedirs(directory)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise


CODE FROM HERE

In [0]:
#DCGAN implementation for MNIST


import torch
from torch import nn
from torch.optim import Adam
from torch.autograd Pimport Variable
from IPython import display
from torchvision import transforms, datasets
import os
import numpy as np
import errno
import torchvision.utils as vutils
from tensorboardX import SummaryWriter
from matplotlib import pyplot as plt


                
#data generator function to create MNIST data
#uses inbuilt dataloader from pytorch datasets , downloads the dataset if necessary 
def data_generator():
    # applies transfroms to each image to convert it to a valid format by resizing and bringing it into a (-1,1) range
    compose = transforms.Compose([
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize((.5,.5,.5),(.5,.5,.5))
    ])
    out_dir='{}/dataset'.format('MNIST')
    return datasets.MNIST(out_dir,True,compose,download=True)

data = data_generator()
batch_size = 100
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
num_batches = len(data_loader)


#discriminator net 
class DiscriminatorNet(torch.nn.Module):
# Network for discriminator is created in this part of code
# Four layers have 1, 128, 256, 512, 1024 and 1 channels respectively.
# Batch Normalisation is applied to  all hidden layers.
# LeakyReLU with negetive slope 0.2 is applied to all the layers except the output layer. 
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        # First convolutional layer takes 1 channel and returns 128 channels, applies a 4*4 kernel with a padding of 1 and stride of 2.
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,128,4,2,1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        )
        # Second convolutional layer takes 128 channel input and returns 256 channels, applies a 4*4 kernel with a padding of 1 and stride of 2.
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 256,4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True)
        )
        # Third convolutional layer takes 256 channels and returns 512 channels, applies a 4*4 kernel with a padding of 1 and stride of 2.
        self.conv3 = nn.Sequential(
            nn.Conv2d(256,512, 4, 2, 1,bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
        )
        # Fourth convolutional layer takes 512 channels and returns 1024 channels,applies a 4*4 kernel with a padding of 1 and stride of 2. 
        self.conv4 = nn.Sequential(
            nn.Conv2d(512, 1024, 4, 2, 1, bias=False),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True)
        )
        # Linear network is used for the last layer.
        # Sigmoid activation function is  used. 
        self.out = nn.Sequential(
            nn.Linear(1024*4*4, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        # Flatten and apply sigmoid
        x = x.view(-1, 1024*4*4)
        x = self.out(x)
        return x

#generatorNet Class contains our generator network which inherits the nn.Module 
class generatorNet(nn.Module):
    def __init__(self):
        super(generatorNet, self).__init__()
        #a liner layer to convert a noise vector of size 100 to a size of 1024*4*4
        self.linear = torch.nn.Linear(100, 1024 * 4 * 4)
        
        #you can view the architecture in a more feasible manner in the image provided
        #ReLu activation has been used in every layer and A batchnorm has been applied after the output of Deconvolutional layer
        
        #first deconvolutional layer which takes 1024 input channels and returns 512 output channels , applies a 4*4 kernel with a padding of 1 and stride of 2.         
        self.deconv1 = nn.Sequential(
            nn.ConvTranspose2d(1024,512, 4, 2, 1, bias=False ),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True)
        )
        
        #second deconvolutional layer which takes 512 input channels and returns 256 output channels , applies a 4*4 kernel with a padding of 1 and stride of 2. 
        self.deconv2 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        
        #fourth deconvolutional layer which takes 256 input channels and returns 128 output channels , applies a 4*4 kernel with a padding of 1 and stride of 2. 
        self.deconv3 = nn.Sequential(
            nn.ConvTranspose2d(256, 128,4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        
        #fifth deconvolutional layer which takes 128 input channels and returns 1 output channel , applies a 4*4 kernel with a padding of 1 and stride of 2.
        #we dont apply batchnorm to this deconvolutional layer.
        self.deconv4 = nn.Sequential(
            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False)
        )
        
        #tanh activator for the output of fifth deconvolutional layer. 
        self.out = torch.nn.Tanh()

    def forward(self, x):
        # Project and reshape
        x = self.linear(x)
        x = x.view(x.shape[0], 1024, 4, 4)
        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.deconv4(x)
        # Apply Tanh
        return self.out(x)


#noise generator takes size as a parameter and generates a noise vector with size 100   
def noise(size):
    n = Variable(torch.randn(size, 100))
    if torch.cuda.is_available(): return n.cuda()
    return n


#weight initialization function to initialize the weights of both the generator and discriminator   
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find('BatchNorm') != -1:
        m.weight.data.normal_(0.00, 0.02)
# Get instance of generator and discriminator network 
generator = generatorNet()
discriminator = DiscriminatorNet()
# Initialise the weights
generator.apply(init_weights)
discriminator.apply(init_weights)

#set it to cuda if cuda enabled GPU is available
if torch.cuda.is_available():
    generator.cuda()
    discriminator.cuda()

# both the networks use Adam optimizer with learning rate of 0.0002 and beta of 0.5 with NO weight decay.
d_optimizer = Adam(discriminator.parameters(),lr=0.0002, betas=(0.5,0.999))
g_optimizer = Adam(generator.parameters(),lr=0.0002,betas=(0.5,0.999))


#A binay cross entropy loss has been used as its very similar to the loss described in the DCGAN paper.
loss = nn.BCELoss()
num_epochs = 50


#this function takes the size as parameter and generates a vector of 1s with the given size , this vector will be our real data target.
def real_data_target(size):
    if torch.cuda.is_available():
        return Variable(torch.ones(size,1)).cuda()
    return Variable(torch.ones(size,1))

#this function takes the size as parameter and generates a vector of 0s with the given size , this vector will be our fake data target.
def fake_data_target(size):
    if torch.cuda.is_available():
        return Variable(torch.zeros(size,1)).cuda()
    return Variable(torch.zeros(size,1))

#training function for the discriminator
#takes input as the optimizer , real image data , and the generated (fake) images data.
def train_discriminator(optimizer, real_data, fake_data):
    #set the gradients to zero before optimization
    optimizer.zero_grad()
    
    #get the prediction from the discriminator for the real data and calculate the error by comparing it to the real data target 
    prediction_real = discriminator(real_data)
    error_real = loss(prediction_real,real_data_target(real_data.size(0)))
    #calculate all the gradients going backward for the disriminator
    error_real.backward()

    #get the prediction from the discriminator for the fake data and calculate the error by comparing it to the fake data target 
    prediction_fake = discriminator(fake_data)
    error_fake = loss(prediction_fake,fake_data_target(real_data.size(0)))
    #calculate all the gradients going backward for the generator 
    error_fake.backward()

     #backpropogate the weight and optimize both the nets
    optimizer.step()

    return error_real+error_fake,prediction_real,prediction_fake
  
  
#training function for the generator
def train_generator(optimizer,fake_data):
    optimizer.step()
    # get predictions from discriminator by feeding fake data
    prediction = discriminator(fake_data)
    # calcuate the error of prediction by comparing it to real data target
    error = loss(prediction,real_data_target(prediction.size(0)))
    # backpropagate the error and update the weights
    error.backward()

    optimizer.step()
    return error

#logger configuratiom.
logger = Logger(model_name='DCGAN', data_name='MNIST')

#generate noise samples for testing.
num_test_samples = 16
test_noise = noise(num_test_samples)

#trainig loop for the number of epochs
for epoch in range(num_epochs):
    #loop through one batch of data in ths loop.
    for n_batch, (real_batch, _) in enumerate(data_loader):

        real_data = Variable(real_batch)
        if torch.cuda.is_available(): real_data = real_data.cuda()
        # Generate fake data
        fake_data = generator(noise(real_data.size(0))).detach()
        # Train the discriminatorNet
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer,
                                                                real_data, fake_data)

        # Generate fake data
        fake_data = generator(noise(real_batch.size(0)))
        # Train Generator 
        g_error = train_generator(g_optimizer, fake_data)
        # Log error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)

        # Display Progress
        if (n_batch) % 10 == 0:
            display.clear_output(True)
            # Display Images
            test_images = generator(test_noise).data.cpu()
            logger.log_images(test_images, num_test_samples, epoch, n_batch, num_batches);
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            )
        # Save the model that has been trained 
        logger.save_models(generator, discriminator, epoch)